In [2]:
import numpy as np
from os.path import dirname, abspath, join as pjoin
from Corrfunc.theory.DD import DD
from Corrfunc.io import read_catalog
from Corrfunc.utils import convert_3d_counts_to_cf
from Corrfunc.mocks import *
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import glob, time 

from astropy.io import fits as fits
from astropy.table import Table, vstack
from astropy import constants as const
from astropy import units as u
from astropy.table import QTable

import healpy as hp
import math

import sys
sys.path.append('/global/homes/l/lbigwood/LSS/py')
import LSS
import LSS.SV3
import LSS.SV3.cattools as cattools

from desitarget.targets import desi_mask, bgs_mask, mws_mask 
from desitarget.geomask import get_imaging_maskbits 

import scipy.stats

In [4]:
#https://arxiv.org/pdf/2106.13120.pdf

to_grab=glob.glob('/global/cfs/cdirs/desi/target/catalogs/dr9/1.1.1/targets/main/resolve/bright/targets-bright-hp-*.fits') 

# very good practice to apply sorted, otherwise the file ordering will be random 	and non-repeatable.  
to_grab = sorted(to_grab) 

hp_stack = []

#do timer as takes a while
start = time.time() 

#total number of pixels, not quite sure where this has come from as npix is less than this above 


mmask = 'BGS_TARGET'
ttype = 'BGS_BRIGHT'

#loop through pixels
for i, x in enumerate(to_grab):
    x = fits.open(x)
    f = np.array(x[1].data)
    #mask for bgs objects
    
    is_bgs = (f[mmask] & bgs_mask[ttype]) != 0
    #idx = np.arange(len(x))[is_bgs]
    #x = x.iloc[idx] 
    hp_stack.append(f[is_bgs])

    #more timing stuff
    if (i % 20) == 0:
        runtime = (time.time() - start)

        print('Runtime of {:.6f} seconds after {:d} pixels'.format(runtime, i))

    if i>40:
        break

data_stack = np.concatenate(hp_stack)      
        

data_stack = Table(data_stack)






data_stack

Runtime of 0.292363 seconds after 0 pixels
Runtime of 8.291325 seconds after 20 pixels
Runtime of 14.980548 seconds after 40 pixels


RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ [5],EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,SCND_TARGET,HPXPIXEL
int16,int32,bytes8,int32,bytes4,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32,float32,float32,float32,int16,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,int8,float32,uint8,float32,float32,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64
9010,357873,0399p047,473,DEV,39.87345241480437,2930574000000.0,4.665933584901227,3185468000000.0,161609.44 .. 181102.81,0.1024539,3.738091,14.036136,30.110819,880.15857,418.30585,94.379974,0.7383891,0.8152202,0.8920133,0.004986856,0.0017506279,0.0011496426,0.0077328524,0.0064884406,0.056162167,0.99624085,0.99661744,0.9897588,6,6,10,2678.795,1905.5867,279.66745,1633.5319,1008.70233,157.55334,45.112095,31.339207,82.80839,1426.1816,2.0547602,0.53359205,0.0008559196,1.4580163e-05,0.98278695,0.9893936,0.9977284,0.99914163,0,0,0,1.6517395,6.202107,13.30498,1.6518486,6.2023234,13.305227,0.0,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.6635122,-0.074558586,-0.09223902,57146.562,37543.81,38002.93,4.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70,0.0,128,0.0,0.0,0.0,0.0,0.0,0.0,S,39627900093202905,1152921504606846976,131074,0,0.36740337142622204,516,2100,2,0,42
9010,357873,0399p047,585,SER,39.877053486508814,1634356400000.0,4.711238350826682,1648187100000.0,645859.75 .. 742408.6,0.10620722,11.396996,31.736794,62.562927,656.30273,344.781,77.72876,0.73023057,0.80914164,0.8882868,0.0010881147,0.0018458276,0.0016095864,0.0037045756,0.003267163,0.0039124205,0.87428254,0.9739295,0.9690705,7,8,10,3180.7239,2323.2947,313.10336,1888.0743,1250.9457,173.8955,64.94275,54.019405,280.97385,-1919.5293,1.7914435,0.48162428,0.0007866988,1.3766008e-05,0.98216206,0.9890072,0.9976453,0.9991102,0,0,0,4.663695,12.986819,25.600994,4.6638255,12.987787,25.602383,2015.5,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.9828502,0.356761,-0.013325343,132656.48,187190.22,119143.54,4.824103,571.145,5991479378248832,G2,20.276392,70.010086,19.972994,22.886183,18.27832,20.094048,3.6527278,13.809634,

In [5]:
#obviously finding the mode of some of these columns is meaningless 

for i in range(len(data_stack.colnames)):
    
    modal = scipy.stats.mode(data_stack[data_stack.colnames[i]])[0]
    print(data_stack.colnames[i],modal)

RELEASE [9011]
BRICKID [364363]
BRICKNAME [b'2278p057']
BRICK_OBJID [886]
MORPHTYPE [b'SER']
EBV [0.02592011]
FLUX_G [4.7808156]
FLUX_R [16.013266]
FLUX_Z [0.]
FLUX_IVAR_G [69.21962]
FLUX_IVAR_R [21.883476]
FLUX_IVAR_Z [0.]
FLUX_W1 [29.812258]
FLUX_W2 [26.986595]
FLUX_IVAR_W1 [1.6599876]
FLUX_IVAR_W2 [0.5373156]
FIBERFLUX_G [2.6638231]
FIBERFLUX_R [4.578155]
FIBERFLUX_Z [0.]
FIBERTOTFLUX_G [1.6523919]
FIBERTOTFLUX_R [3.1275606]
FIBERTOTFLUX_Z [16.833574]
MASKBITS [0]
SHAPE_R [0.]
SHAPE_E1 [0.]
SHAPE_E2 [0.]
SERSIC [1.]
REF_ID [0]
REF_CAT [b'']
GAIA_PHOT_G_MEAN_MAG [0.]
GAIA_PHOT_BP_MEAN_MAG [0.]
GAIA_PHOT_RP_MEAN_MAG [0.]
PHOTSYS [b'N']
TARGETID [39627785479655281]
RA [22.53458708]
DEC [0.01997033]
REF_EPOCH [0.]
PARALLAX [0.]
PMRA [0.]
PMDEC [0.]


In [8]:
#if sticking strictly to columns at '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/debug_jl/MinExampleDir/alt_mtls_sandbox/Univ000/fa/SV3/20210408/000057-targ.fits'

#this is if use this targeting file for columns 

dat = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/debug_jl/MinExampleDir/alt_mtls_sandbox/Univ000/fa/SV3/20210408/000057-targ.fits')
dat.pprint()

for x in dat.dtype.names:
    print(x)



for i in range(len(dat.dtype.names)):
    if dat.dtype.names[i] =='SV3_DESI_TARGET':
        break
    
    modal = scipy.stats.mode(data_stack[dat.dtype.names[i]])[0]
    print(dat.dtype.names[i],modal)

RELEASE [9011]
BRICKID [364363]
BRICKNAME [b'2278p057']
BRICK_OBJID [886]
MORPHTYPE [b'SER']
EBV [0.02592011]
FLUX_G [4.7808156]
FLUX_R [16.013266]
FLUX_Z [0.]
FLUX_IVAR_G [69.21962]
FLUX_IVAR_R [21.883476]
FLUX_IVAR_Z [0.]
FLUX_W1 [29.812258]
FLUX_W2 [26.986595]
FLUX_IVAR_W1 [1.6599876]
FLUX_IVAR_W2 [0.5373156]
FIBERFLUX_G [2.6638231]
FIBERFLUX_R [4.578155]
FIBERFLUX_Z [0.]
FIBERTOTFLUX_G [1.6523919]
FIBERTOTFLUX_R [3.1275606]
FIBERTOTFLUX_Z [16.833574]
MASKBITS [0]
SHAPE_R [0.]
SHAPE_E1 [0.]
SHAPE_E2 [0.]
SERSIC [1.]
REF_ID [0]
REF_CAT [b'']
GAIA_PHOT_G_MEAN_MAG [0.]
GAIA_PHOT_BP_MEAN_MAG [0.]
GAIA_PHOT_RP_MEAN_MAG [0.]
PHOTSYS [b'N']
TARGETID [39627785479655281]
RA [22.53458708]
DEC [0.01997033]
REF_EPOCH [0.]
PARALLAX [0.]
PMRA [0.]
PMDEC [0.]
